In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt

# vectorization
from sklearn.preprocessing import LabelEncoder

# metrics
from sklearn.metrics import accuracy_score

# misc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

# models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [6]:
# load train and test data

train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

## 1. Structure of the Data

Size of **training examples**: (891,12) <br/>
Size of **test data**: (418,11)

In [12]:
print(f'Train dataset size: {train.shape}')
train.head()

Train dataset size: (891, 13)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [ ]:
print(f'Test dataset size: {test.shape}')
test.head()

## 2. Missing Values

In [13]:
def missing_values(df, columns):
    """ Finds number of rows where specified columns are missing values.
    
    Args:
        df:
            The dataframe to be analyzed.
        columns:
            The list of columns of the dataframe.
    
    Returns:
         A dictionary with the columns and the number of values they are missing.
    """
    missing = {}
    
    for column in columns:
        total = df[column].value_counts().sum()
        missing[column] = df.shape[0] - total
    
    return missing

In [14]:
missing = missing_values(train, train.columns)
print(f'Total values that each column should have: {train.shape[0]}')
print(f'Total values that each column is missing:')
missing

Total values that each column should have: 891
Total values that each column is missing:


{'PassengerId': 0,
 'Survived': 0,
 'Pclass': 0,
 'Name': 0,
 'Sex': 0,
 'Age': 0,
 'SibSp': 0,
 'Parch': 0,
 'Ticket': 0,
 'Fare': 0,
 'Cabin': 687,
 'Embarked': 2,
 'Title': 1}

**Observations:**
* The **PassengerId** and **Name** columns have unique values for each row in the trainning examples. Thus, they won't help in classification. <br/>
* We can create an new feature, called **Title** which will be the title of each person.
* The **Age** column is missing a lot of values. We'll fill them using the mean of the ages with respect to the persons title . <br/>
* The **Cabin** column has a lot of missing values. Specifically, it is missing 687/891. For now, we'll drop the column. <br/>
* The **Embarked** column has 2 missing values. So, we'll remove these 2 rows.

**Notes:**.
* Can I group the values in cabin column?

## 3. Number of Unique Values

In [ ]:
print(f'Ticket column has {len(train.Ticket.unique())} different values.')

In [ ]:
print(f'Fare column has {len(train.Fare.unique())} different values.')

**Observations:**
* The **Ticket** column has a lot of different values. Specifically, 681/891 different values. For now, we'll drop this column.
* The **Fare** column has a quite many different values. We'll create ranges.

**Notes:**
* Can I group together the different values in ticket column?

## 4. Preprocess

In [7]:
# create a column with the titles

train['Title'] = train['Name'].str.extract(', (\w+)\.', expand=False)

In [9]:
# insert missing data in "Age" column using mean with respect to the "Title" column

train['Age'] = train['Age'].fillna(train.groupby('Title')['Age'].transform('median'))

In [15]:
# remove rows

train = train.dropna(subset=['Embarked'])

In [16]:
# create bins for "Fare" and "Age" (discretise)

train['Fare'] = pd.cut(train['Fare'], bins=5)
train['Age'] = pd.cut(train['Age'], bins=10)

In [18]:
# remove columns

train = train.drop(['PassengerId', 'Name' ,'Ticket', 'Cabin'], axis=1)

KeyError: "['PassengerId' 'Name' 'Ticket' 'Cabin'] not found in axis"

In [19]:
print('Training examples after preprocessing:')
train.head()

Training examples after preprocessing:


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,"(16.336, 24.294]",1,0,"(-0.512, 102.466]",S,Mr
1,1,1,female,"(32.252, 40.21]",1,0,"(-0.512, 102.466]",C,Mrs
2,1,3,female,"(24.294, 32.252]",0,0,"(-0.512, 102.466]",S,Miss
3,1,1,female,"(32.252, 40.21]",1,0,"(-0.512, 102.466]",S,Mrs
4,0,3,male,"(32.252, 40.21]",0,0,"(-0.512, 102.466]",S,Mr


## 5. Vectorize

We'll try categorical encoding vectors.

In [20]:
train_enc_cat = train.apply(LabelEncoder().fit_transform)

In [21]:
print(f'Categorical encoding vector size: {train_enc_cat.shape}')
train_enc_cat.head()

Categorical encoding vector size: (889, 9)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,2,1,2,1,0,0,2,11
1,1,0,0,4,1,0,0,0,12
2,1,2,0,3,0,0,0,2,8
3,1,0,0,4,1,0,0,2,12
4,0,2,1,4,0,0,0,2,11


## 6. Fitting Models and Score

In [22]:
y_train = train_enc_cat.Survived
X_train = train_enc_cat.drop(['Survived'], axis=1)

In [23]:
print(f'Train data size: {X_train.shape}')
X_train.head()

Train data size: (889, 8)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,2,1,2,1,0,0,2,11
1,0,0,4,1,0,0,0,12
2,2,0,3,0,0,0,2,8
3,0,0,4,1,0,0,2,12
4,2,1,4,0,0,0,2,11


In [24]:
print(f'Train labels size: {y_train.shape}')
y_train.head()

Train labels size: (889,)


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

### 6.1 Logistic Regression

In [25]:
model = LogisticRegression()

# cross validation
y_pred = cross_val_predict(model, X_train, y_train, cv=10)

# accuracy
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy of training examples: {accuracy}')

Accuracy of training examples: 0.7941507311586051


### 6.2 Decision Tree

In [ ]:
model = DecisionTreeClassifier()

# cross validation
y_pred = cross_val_predict(model, X_train, y_train, cv=10)

# accuracy
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy for Decision Tree after cross validation: {accuracy}')

### 6.3 Random Forest

In [ ]:
model = RandomForestClassifier(max_depth=2)

# cross validation
y_pred = cross_val_predict(model, X_train, y_train, cv=10)

# accuracy
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy for Random Forest after cross validation: {accuracy}')

## 7. Predict on Test Data

We'll perform the same transformations to the test data.

### 7.1 Preprocessing

In [30]:
# create a column with the titles
test['Title'] = test['Name'].str.extract(', (\w+)\.', expand=False)

# insert missing data in "Age" column using mean with respect to the "Title" column
test['Age'] = test['Age'].fillna(test.groupby('Title')['Age'].transform('median'))

# create bins for "Fare" and "Age" (discretise)
test['Fare'] = pd.cut(test['Fare'], bins=5)
test['Age'] = pd.cut(test['Age'], bins=10)

In [32]:
# test data should have the same format (columns) as training examples
columns = X_train.columns

In [26]:
# test data should have the same format (columns) as training examples
columns = X_train.columns

### 7.2 Vectorize

In [33]:
test_enc_cat = test[columns].apply(LabelEncoder().fit_transform)

### 7.3 Predict with Best Model

Best model up to now is logistic regression classifier, with training accuracy 77%.

In [ ]:
X_test = test_enc_cat

In [ ]:
best_model = LogisticRegression()

# train model
model.fit(X_train, y_train)

# predict on test data
y_pred = model.predict(X_test)

## 8 Submission

In [ ]:
submission = pd.DataFrame()
submission['PassengerId'] = test['PassengerId']
submission['Survived'] = y_pred

In [ ]:
print(f'Submission size: {submission.shape}')
submission.head()

In [ ]:
submission.to_csv('/kaggle/working/decision-tree.csv', index=False)